In [9]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

Creating RDD

In [2]:
ord = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Orders')
ord_items = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Order_items')

#### sortByKey(ascending=True, numPartitions=None, keyfunc=<function <lambda>>)
When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in
the Boolean ascending argument.

Example : Sort ordes using customer id.

In [3]:
ord.map(lambda x : (int(x.split(',')[2]),x)).sortByKey(ascending=True).take(5)

[(1, '22945,2013-12-13 00:00:00.0,1,COMPLETE'),
 (2, '15192,2013-10-29 00:00:00.0,2,PENDING_PAYMENT'),
 (2, '33865,2014-02-18 00:00:00.0,2,COMPLETE'),
 (2, '57963,2013-08-02 00:00:00.0,2,ON_HOLD'),
 (2, '67863,2013-11-30 00:00:00.0,2,COMPLETE')]

Example (Sort orders using customer and status)

In [4]:
ordPair = ord.map(lambda x : ((int(x.split(',')[2]), x.split(',')[3]),x))
ordSort = ordPair.sortByKey(ascending=False)
for i in ordSort.take(10) : print(i)

((12435, 'PENDING'), '41643,2014-04-08 00:00:00.0,12435,PENDING')
((12435, 'CANCELED'), '61629,2013-12-21 00:00:00.0,12435,CANCELED')
((12434, 'PENDING_PAYMENT'), '4799,2013-08-23 00:00:00.0,12434,PENDING_PAYMENT')
((12434, 'PENDING'), '5303,2013-08-26 00:00:00.0,12434,PENDING')
((12434, 'PENDING'), '13544,2013-10-16 00:00:00.0,12434,PENDING')
((12434, 'ON_HOLD'), '51800,2014-06-14 00:00:00.0,12434,ON_HOLD')
((12434, 'COMPLETE'), '6160,2013-09-02 00:00:00.0,12434,COMPLETE')
((12434, 'COMPLETE'), '42915,2014-04-16 00:00:00.0,12434,COMPLETE')
((12434, 'COMPLETE'), '61777,2013-12-26 00:00:00.0,12434,COMPLETE')
((12434, 'CLOSED'), '1868,2013-08-03 00:00:00.0,12434,CLOSED')


#### Global Ranking
- sortByKey and take
- takeOrdered or top


#### Ranking Per Group:
- Getting ranking per group is a bit complex but important to know.
- Per key or Per group ranking can be achieved using
    - groupByKey with flatMap
    - Python Knowledge like sorted function, list etc

Example: Top five products with highest prices.

In [5]:
rdd_products = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Products')
rdd_products.take(5)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat",
 "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 '5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet']

In [6]:
rdd_products = rdd_products.filter(lambda x : x.split(",")[4]!="")
pair_rdd = rdd_products.map(lambda x : (float(x.split(',')[4]),x))
pair_rdd.sortByKey(ascending=False).take(5)

[(1999.99,
  '208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical'),
 (1799.99,
  '66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill'),
 (1799.99,
  '199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill'),
 (1799.99,
  '496,22,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill'),
 (1099.99,
  '1048,47,"Spalding Beast 60"" Glass Portable Basketball ",,1099.99,http://images.acmesports.sports/Spalding+Beast+60%22+Glass+Portable+Basketball+Hoop')]

Using takeOrdered

In [7]:
rdd_products.takeOrdered (5,lambda k: -float( k.split (',')[4]))

['208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical',
 '66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '496,22,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '1048,47,"Spalding Beast 60"" Glass Portable Basketball ",,1099.99,http://images.acmesports.sports/Spalding+Beast+60%22+Glass+Portable+Basketball+Hoop']

Example: Top 2 Products with highest Prices per Category.

In [8]:
prod_df = rdd_products.filter(lambda x : (int(x.split (',')[1]) in [2,3,4]) and (int(x.split (',')[0]) in [1,2,3,4,5,25,26,27,28,29,49,50,51,52,53]))

prod_groupby = prod_df.map(lambda line : (int(line.split (',')[1]), line)).groupByKey()
first = prod_groupby.first()
sorted(first[1],key = lambda x : float(x.split (',')[4]),reverse=True)
prod_groupby.flatMap(lambda x: sorted(x[1], key=lambda k:float(k.split(",")[4]), reverse=True)[:2]).collect()

['5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat",
 '49,4,Diamondback Adult Sorrento Mountain Bike 2014,,299.98,http://images.acmesports.sports/Diamondback+Adult+Sorrento+Mountain+Bike+2014',
 '52,4,Easton Mako Youth Bat 2014 (-11),,249.97,http://images.acmesports.sports/Easton+Mako+Youth+Bat+2014+%28-11%29',
 "26,3,Nike Men's USA White Home Stadium Soccer Jers,,90.0,http://images.acmesports.sports/Nike+Men%27s+USA+White+Home+Stadium+Soccer+Jersey",
 "29,3,Nike Men's USA Away Stadium Replica Soccer Je,,90.0,http://images.acmesports.sports/Nike+Men%27s+USA+Away+Stadium+Replica+Soccer+Jersey"]

#### union(other)
- A union will get all the elements from both the data sets.
- In the case of a union, it will not get distinct elements. Apply distinct, if you only want to get distinct elements after un ion
operation.
- When we use set operations such as union and intersect, data should have a similar structure (Same Columns and Types).

Example: Number of customers placed order in July or Aug Month.

In [24]:
orders_july = ord.filter(lambda x : x.split(',')[1].split("-")[1]=='07').map(lambda x : x.split(',')[2])
orders_ago = ord.filter(lambda x : x.split(',')[1].split("-")[1]=='08').map(lambda x : x.split(',')[2])

orders_july.union(orders_ago).distinct().count()

7633

#### Intersection(other)
- Return the intersection of this RDD and another one.
- The output will not contain any duplicate elements, even if the input RDDs did.

Example: Orders applied both in July and Aug datamonth

In [26]:
orders_july.intersection(orders_ago).count()

1759

Example: Check if duplicates are reported.

In [29]:
rdd1=sc.parallelize ([1, 2, 3, 3, 3])    
rdd2=sc.parallelize ([1, 3, 5])
rdd1.intersection(rdd2).collect()

[1, 3]

#### sample(withReplacement , fraction, seed=None)
- To get random sample records from the RDD.
- withReplacment : True or False. With True, Same result can be produced more than
- Fraction: Between 0 to 1. 0.3 means 30%. Does not guarantee the exact 30% of the records.
- Seed: Reproduce the same sample.

In [65]:
rdd= sc.parallelize (range(100),4)
rdd.sample(seed=10,fraction=0.1,withReplacement=False).collect()

[5, 19, 20, 36, 37, 46, 47, 51, 65, 70, 72, 77, 96]

#### takeSample(withReplacement , num , seed=None)
- Return a fixed size sampled subset of this RDD.
- This method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver's m emo ry.
- num is exact

In [66]:
rdd.takeSample(seed=10,num=10,withReplacement=True)

[55, 73, 89, 9, 33, 60, 74, 11, 64, 85]